In [5]:
!pip install beautifulsoup4

     |████████████████████████████████| 115 kB 5.2 MB/s eta 0:00:01


In [3]:
import requests
import bs4
import pandas as pd
import shutil


from tqdm import tqdm

In [4]:
results = []
for i in tqdm(range(1, 26)):
    res = requests.get('https://www.newyorker.com/culture/cover-story/page/%s' % i)
    res_bs4 = bs4.BeautifulSoup(res.text)
    fechas = res_bs4.find_all('h6')
    links = [item["srcset"].split(', ')[-1][:-3] for item in res_bs4.find_all('source', attrs={'media': "(min-width: 1280px)"})]
    
    results.extend(list(zip(fechas, links)))

100%|██████████| 25/25 [00:05<00:00,  4.37it/s]


In [5]:
pd_results = pd.DataFrame(results, columns=['date', 'link'])

In [6]:
pd_results['date'] = pd_results['date'].apply(lambda x: x.text)

In [7]:
pd_results['date'] = pd.to_datetime(pd_results['date'])

In [8]:
pd_results.to_csv('manifest.txt')

In [9]:
pd_results['link'].str.split('/').apply(lambda x: x[-1])

0                Abney_Tilley_Covid_Dining_promo.jpg
1             cover-PROMO-image-gurbuz_reopening.jpg
2      cover-promo_image-kadir_nelson_homecoming.jpg
3          cover-PROMO-image-ulriksen-basketball.jpg
4              cover-PROMO-mccall-tech-chrystler.jpg
                           ...                      
231                         120903_2012_p465_opt.jpg
232                             120618_2012_p465.jpg
233                             120130_2012_p465.jpg
234          2011_11_28_Niemann_Pilgrims%2520(1).jpg
235                             111107_2011_p465.jpg
Name: link, Length: 236, dtype: object

In [10]:
for date, link in results:
    new_date = str(pd.to_datetime(date.text))
    response = requests.get(link, stream=True)
    
    with open('%s.png' % new_date, 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)